# Классификация на основе логит-модели множественного выбора

### Задачи

1. **Исследовать возможность прогнозирования класса кредитоспособности новых наблюдений на основе логит-модели множественного выбора.**
2. **Сравнить результаты классификации на основе логит-модели множественного выбора с результатами классификации на основе дискриминантного анализа.**

In [1]:
import sys
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from statsmodels.miscmodels.ordinal_model import OrderedModel

sys.path.append('../src')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from discr_analysis_utils import create_cm_without_test, deviation_distribution

In [2]:
file_path = '../data/data_clustered.csv'
df = pd.read_csv(file_path)

display(df)

,I,F1,F2,F3,F4,F5,k1,k2,k4,k5,k6,k7,k9,k10,k11,k13,k14,k15,k18,k19,Cluster
0,-0.158751,-0.314829,0.829452,-1.069358,-0.026382,-0.460131,0.000000,0.014084,0.000000,0.797945,0.449936,0.000000,1.000000,0.000000,0.012880,0.062375,0.383869,0.795137,0.002558,0.002752,3
1,0.423813,1.042911,1.147481,-0.177917,0.021606,0.080521,0.245124,0.299176,0.558148,0.910623,0.264507,1.000000,1.000000,0.000000,0.028820,0.082380,1.000000,1.000000,0.003869,0.002987,1
2,-0.333018,-0.612924,1.103026,-0.856950,-0.730159,-1.320378,0.000000,0.000000,0.000000,0.764261,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000528,3
3,0.801346,2.530072,-0.508093,1.008972,0.124333,0.568267,0.809119,0.453227,0.853955,0.940561,0.899900,0.962963,1.000000,0.339213,0.024809,0.714604,1.000000,1.000000,0.005724,0.011632,1
4,-0.167511,0.808507,0.459595,-1.647264,-0.807870,-1.351857,0.194844,0.000000,0.444533,0.864509,0.000000,0.000000,1.000000,0.007876,0.000000,0.000000,0.514314,0.625071,0.002160,0.001286,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,0.334152,0.772825,-0.263806,0.868460,1.244943,-0.826719,0.323204,0.142995,0.397917,0.757061,0.041652,0.874880,0.795362,0.355109,0.051856,0.419410,0.829911,1.000000,0.004824,0.011298,1
2691,0.330281,-0.115035,1.252717,0.646925,0.524918,0.389984,0.143047,0.151816,0.157581,0.830148,0.354400,1.000000,1.000000,0.006451,0.066174,0.380569,1.000000,1.000000,0.001887,0.004243,2
2692,0.173085,-0.099070,0.455273,1.861309,0.615917,-1.803509,0.113393,0.000000,0.190636,0.808592,0.000000,0.000000,0.836614,0.050279,0.045358,0.714604,1.000000,1.000000,0.002285,0.009967,3
2693,0.182451,-0.811812,1.148944,0.794026,0.600394,0.612646,0.077662,0.000000,0.000000,0.748261,0.494889,0.907597,0.868915,0.033910,0.074567,0.439297,0.944861,1.000000,0.001407,0.003355,2


**В итоговую модель вошло 7 финансовых коэффициентов. Все они являются статистически значимыми на уровне 0.05**

In [3]:
k = ['k1', 'k6', 'k7', 'k9', 'k10', 'k13', 'k18']
X = df[k]
y = df['Cluster']

**В работе используется логит-модель множественного выбора с упорядоченной зависимой переменной (ordered logit). Применение упорядоченной логит-модели обусловлено наличием естественного порядка между категориями зависимой переменной.**

**Пседво R^2 используется для оценки моделей логистической регрессии и показывает степень улучшения модели по сравнению с константной.**

**LR test показывает, насколько хорошо модель описывает наблюдаемые данные, а LR-критерий и соответствующее p-значение используются для проверки общей статистической значимости модели.**

**Информационные критерии Акаике (AIC) и Шварца (SC/BIC) — это статистические критерии, используемые для сравнения и выбора наилучшей из нескольких моделей, описывающих одни данные; меньшие значения AIC и SC соответствуют более предпочтительной модели.**

In [4]:
model = OrderedModel(y, X, distr='logit')
result = model.fit(method='bfgs', disp=False)

print(f"Pseudo R^2: {result.prsquared:.3f}")
print(f"LR p-value: {result.llr_pvalue:.4f}")
print(f"AIC: {result.aic:.2f}")
print(f"SC: {result.bic:.2f}")

Pseudo R^2: 0.499
LR p-value: 0.0000
AIC: 3720.17
SC: 3779.16


**Гипотеза об адекватности модели в целом не отклоняется на основании критерия отношения правдоподобия (P-значение для LR-статистики менее 0.05), значение статистического критерия pseudo R^2 равно 0.499. Информационные статистики Акаике и Шварца равны: AIC=3720.17, SC=3779.16.**

## Результаты оценки модели множественного выбора

In [5]:
variables_df = pd.DataFrame({
    'Переменная': k,
    'Коэффициент': [result.params[i] for i in range(7)],
    'z-статистика': [result.tvalues[i] for i in range(7)],
    'P-значение': [result.pvalues[i] for i in range(7)]
})

thresholds_df = pd.DataFrame({
    'Переменная': ['C(1)', 'C(2)', 'C(3)'],
    'Коэффициент': [result.params[i] for i in range(7, 10)],
    'z-статистика': [result.tvalues[i] for i in range(7, 10)],
    'P-значение': [result.pvalues[i] for i in range(7, 10)]
})

display(variables_df.round(6).style.hide())

display(thresholds_df.round(6).style.hide())

Переменная,Коэффициент,z-статистика,P-значение
k1,-5.868985,-21.287636,0.000000
k6,-1.007780,-7.720730,0.000000
k7,-2.757997,-19.634827,0.000000
k9,-5.853314,-26.040610,0.000000
k10,1.906919,9.181741,0.000000
k13,-4.048236,-17.724388,0.000000
k18,-79.451577,-2.995456,0.002740


Переменная,Коэффициент,z-статистика,P-значение
C(1),-12.314419,-37.057584,0.000000
C(2),1.348287,40.526655,0.000000
C(3),1.209817,35.311938,0.000000


**z-статистика характеризует степень влияния фактора на зависимую переменную и используется для проверки значимости коэффициента.**

**P-значение показывает статистическую значимость коэффициента модели; при значениях p-value меньше 0.05 соответствующий фактор считается значимым.**

**Знаки коэффициентов в построенной модели согласуются с априорными представлениями. В соответствии с правилами проводимой нормировки преобразованные коэффициенты имеют интерпретацию: чем больше, тем лучше финансовое состояние предприятия. В то же время наиболее благоприятным является минимальное значение рейтинга, поэтому ожидаемым было получение отрицательных знаков при оценках коэффициентов модели (обратная зависимость между номером рейтинга и финансовыми показателями после нормировки).**

## Классификационная таблица, %

In [6]:
y_pred = result.model.predict(result.params, exog=X)
y_pred_class = y_pred.argmax(axis=1) + 1

cm = confusion_matrix(y, y_pred_class)
class_sizes = cm.sum(axis=1)

percent_table = create_cm_without_test(
    cm/class_sizes.reshape(-1,1)*100, 
    (cm/class_sizes.reshape(-1,1)*100).sum(axis=1)
)

display(percent_table)

,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Всего наблюдений
Истинный 1,71.7,28.1,0.2,0.0,100.0
Истинный 2,11.7,65.6,22.5,0.2,100.0
Истинный 3,0.1,24.1,59.8,16.0,100.0
Истинный 4,0.0,0.3,23.9,75.7,100.0


**В соответствии с полученными результатами доля совпадений по отдельным классам колеблется от 59.8% до 75.7%. Наиболее точным оказывается классификация наблюдений, которым соответствуют рейтинги, равные 1 и 4. Плохая разделимость 2 и 3 классов кредитоспособности приводит к более низким показателям точности классификации для наблюдений из этих классов.**

## Ошибки прогнозов, %

In [7]:
deviation_dist = deviation_distribution(y, y_pred_class)

deviation_summary_table = pd.DataFrame({
    "Логит-модель":  [deviation_dist[k] for k in range(-3, 4)] + [deviation_dist['|k|≤1']]
}, index=[-3, -2, -1, 0, 1, 2, 3, '|k|≤1'])

display(deviation_summary_table.T.round(2))

,-3,-2,-1,0,1,2,3,|k|≤1
Логит-модель,0.0,0.11,15.81,67.53,16.44,0.11,0.0,99.78


**В целом, доля точных прогнозов находится на уровне 67.53 %, доля прогнозов с отклонением не более чем на один рейтинговый класс от истинного рейтинга – более 99%. Использование дискриминантного анализа позволяет получить большее количество точных прогнозов. При этом доля прогнозов с отклонением не более чем на один рейтинговый класс от истинного рейтинга в случае дискриминантного анализа немного ниже.**